# 7506 - Organizacion de Datos - TP N°2

# 1. Librerias

In [26]:
#librerías, no es necesario volverlas a importar

import os
import pandas as pd

import nltk, re, string, collections
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

data_loader = os.path.join('..','0_Data','0_DataLoader.ipynb')
#Vectorizacion
%run $data_loader

[nltk_data] Downloading package punkt to /home/celeste/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/celeste/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/celeste/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# 2. Datos de entrenamiento

In [27]:
data = get_data_original_as_np_array()

x_train_original = data.x_train
y_train = data.y_train

x_val_original = data.x_val
y_val_original = data.y_val

data_prep = get_data_sentiment_analysis_as_np_array()

x_train_preprocesado = data_prep.x_train

x_val_preprocesado = data_prep.x_val
y_val_preprocesado = data_prep.y_val


# 3. Datos de test

In [28]:
x_test_original = data.predict

x_test_preprocesado = data_prep.predict

# 4. Vectorizacion

## 4.1  Obtencion de NGramas, análisis de distintas herramientas

In [29]:
def get_ngrams(documents, size):
    word_vectorizer = CountVectorizer()
    sparse_matrix = word_vectorizer.fit_transform(documents)
    frequencies = sum(sparse_matrix).data
    #frequencies = sum(sparse_matrix).toarray()[0]
    df = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency'])
    df = df.sort_values(by='frequency', ascending=False)
    return df

In [30]:
%time get_ngrams(x_train_preprocesado,1)

CPU times: user 1.11 s, sys: 8.01 ms, total: 1.12 s
Wall time: 1.12 s


,frequency
killsåêon,2120
km,1303
kenneth,1239
knight,1188
leak,920
...,...
duda,1
duck,1
dubai,1
dual,1


In [52]:
def documentNgrams(documents, size, count=None):
    ngrams_all = []
    for document in documents:
        tokens = document.split()
        if len(tokens) < size:
            continue
        else:
            output = list(ngrams(tokens, size))
        for ngram in output:
            ngrams_all.append(" ".join(ngram))
    cnt_ngram = Counter()
    for word in ngrams_all:
        cnt_ngram[word] += 1
    df = pd.DataFrame.from_dict(cnt_ngram, orient='index').reset_index()
    df = df.rename(columns={'index':'words', 0:'count'})
    df = df.sort_values(by='count', ascending=False)
    if (count == None):
        return df
    else:
        return(df.head(count))

In [32]:
#elijo este que da mejor tiempo de ejecucion y ademas el otro saltea un par de casos. 
# mas adelante se analiza que los casos que saltea son los de letras solas, por ejemplo "u are" 

%time documentNgrams(x_train_preprocesado,1, 100)

CPU times: user 71.2 ms, sys: 12 µs, total: 71.2 ms
Wall time: 70.2 ms


,words,count
65,the,2117
123,a,1481
17,in,1303
121,to,1238
11,of,1183
...,...,...
1047,were,88
670,face,88
150,storm,87
896,her,86


## 4.2  Obtencion de los 100 BiGramas mas frecuentes para todos los tweets

In [33]:
%time bigrams_all_tweets = documentNgrams(x_train_preprocesado,2, 10000)

CPU times: user 72.5 ms, sys: 4 µs, total: 72.5 ms
Wall time: 72.3 ms


In [34]:
bigrams_all_tweets.head(50)

,words,count
475,in the,215
495,i m,158
115,of the,158
214,it s,129
1077,don t,121
316,on the,85
1447,to the,84
166,like a,79
420,in a,75
1510,blow a,69


In [35]:
bigrams_list = bigrams_all_tweets.words.to_list()

In [36]:
word_vectorizer_100 = CountVectorizer(ngram_range=(2, 2), vocabulary = bigrams_list)
sparse_matrix = word_vectorizer_100.fit_transform(x_train_preprocesado.head(10))

In [37]:
print (sparse_matrix.toarray()[1][31])

0


In [38]:
bigrams_list[31]

'mass murder'

In [39]:
x_train_preprocesado.iloc[1]

'famili member of osama bin laden have die in an airplan accid how iron mhmmm gov shit i suspect'

## 4.3  Obtencion de los 1000 BiGramas mas frecuentes para todos los tweets

In [40]:
%time bigrams_all_tweets_1000 = documentNgrams(x_train_preprocesado,2, 1000)

CPU times: user 141 ms, sys: 0 ns, total: 141 ms
Wall time: 141 ms


In [41]:
bigrams_list_1000 = bigrams_all_tweets_1000.words.to_list()

In [42]:
word_vectorizer_1000 = CountVectorizer(ngram_range=(2, 2), vocabulary = bigrams_list_1000)

In [43]:
vectorizer = CountVectorizer(ngram_range = (2,2), max_features = 5) 
vectorizer.fit_transform(x_train_preprocesado)

<4872x5 sparse matrix of type '<class 'numpy.int64'>'
	with 598 stored elements in Compressed Sparse Row format>

In [44]:
vectorizer.get_feature_names()

['cheeki play', 'in the', 'of the', 'on the', 'to the']

## 4.4  Obtencion de los BiGramas mas frecuentes para los tweets que son desastres 

In [51]:
true_tweets = pd.DataFrame(x_train_preprocesado)
true_tweets = true_tweets.loc[y_train == 1,'texto_preprocesado_sentiment_analysis']
true_tweets

id
203      famili member of osama bin laden have die in a...
4778     trafford centr film fan angri after odeon cine...
3879     deton fashion mountain electron watch water re...
4692     men escap car engulf in flame in parley s cany...
10763    the first piec of wreckag from the first ever ...
                               ...                        
8408                                     sandstorm woo hoo
736      christian attack by muslim at the templ mount ...
6254           the murder stori of americaûª first hijack
3745     ford truck start up and drive off after be des...
3933     abcnew obama declar disast for typhoon devast ...
Name: texto_preprocesado_sentiment_analysis, Length: 2094, dtype: object

In [53]:
%time bigrams_true_tweets = documentNgrams(true_tweets,2)

CPU times: user 47.1 ms, sys: 0 ns, total: 47.1 ms
Wall time: 47.1 ms


In [54]:
bigrams_true_tweets

,words,count
241,in the,99
315,of the,71
763,play or,48
765,blow a,48
764,or blow,48
...,...,...
7462,excus just,1
7463,just like,1
7464,like the,1
7465,whole tooth,1


## 4.5  Obtencion de los TriGramas mas frecuentes para los tweets que son desastres 

In [55]:
%time trigrams_true_tweets = documentNgrams(true_tweets,3)

CPU times: user 56.2 ms, sys: 0 ns, total: 56.2 ms
Wall time: 56.3 ms


In [56]:
trigrams_true_tweets

,words,count
727,or blow a,48
726,play or blow,48
725,cheeki play or,48
724,out cheeki play,48
723,stick out cheeki,48
...,...,...
7686,have drop atom,1
7685,to have drop,1
7684,one to have,1
7683,onli one to,1


## 4.6  Obtencion del vocabulario de los tweets que son desastres naturales  

In [57]:
%time vocabulary_true_tweets = documentNgrams(true_tweets,1)

CPU times: user 39.9 ms, sys: 0 ns, total: 39.9 ms
Wall time: 40 ms


In [58]:
vocabulary_true_tweets

,words,count
57,the,868
8,in,748
88,a,637
2,of,607
86,to,446
...,...,...
3009,com,1
3013,lapseconfus,1
3017,narrowli,1
3022,rabaa,1


In [ ]:
#true_tweets = pd.DataFrame(x_train_original)
#true_tweets = true_tweets.loc[y_train == 1,'text']
#true_tweets

In [ ]:
#vocabulary_true_tweets_original = documentNgrams(df_train.loc[(df_train['target']== 1),'text'],1)

In [ ]:
#vocabulary_true_tweets_original